# Imports

In [ ]:
# import pyspark functions
from pyspark.sql.functions import *
# import URL processing
import urllib

# AWS credentials

In [ ]:
# list tables in filestore to get name of credentials file
dbutils.fs.ls("/FileStore/tables")

Out[2]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000)]

In [ ]:
# Read the CSV file to spark dataframe, passing in options for the header row and separator
aws_keys_df = spark.read.format("csv")\
.option("header", "true")\
.option("sep", ",")\
.load("/FileStore/tables/authentication_credentials.csv")

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secret key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# Mount S3 bucket

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-1215be80977f-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-1215be80977f-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[5]: True

In [ ]:
# list the topics stored on the mounted S3 bucket
display(dbutils.fs.ls("/mnt/user-1215be80977f-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-1215be80977f-bucket/topics/1215be80977f.geo/,1215be80977f.geo/,0,1692041344013
dbfs:/mnt/user-1215be80977f-bucket/topics/1215be80977f.pin/,1215be80977f.pin/,0,1692041344013
dbfs:/mnt/user-1215be80977f-bucket/topics/1215be80977f.user/,1215be80977f.user/,0,1692041344013


# Read contents of S3 bucket into dataframes

In [ ]:
# list of topic suffixes
topics = [".pin", ".geo", ".user"]

def read_topics_into_dataframe(topic):
    # create path to topic files
    file_path = f"/mnt/user-1215be80977f-bucket/topics/1215be80977f{topic}/partition=0/*.json"
    # specify file type
    file_type = "json"
    # Ask Spark to infer the schema
    infer_schema = "true"
    # load JSONs from mounted S3 bucket to Spark dataframe
    df = spark.read.format(file_type) \
        .option("inferSchema", infer_schema) \
        .load(file_path)
    return df

for item in topics:
    # create statement strings for naming and displaying dataframes
    make_df_statement = f"df_{item[1:]} = read_topics_into_dataframe('{item}')"
    display_df_statement = f"display(df_{item[1:]})"
    # execute statements
    exec(make_df_statement)
    exec(display_df_statement)


category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
travel,"This Costa Rica itinerary is the ultimate guide to spending two weeks in Costa Rica. Find out about visiting La Fortuna, Arenal, Monteverde, Naranjo, Corcovado National Park, Or…",1,10k,https://i.pinimg.com/originals/30/93/cb/3093cb01d9de2d125fda8ba5e3e41946.jpg,10138,image,"Wanderlust Chloe ✈️ Travel guides, inspo and adventure travel ✈️",Local save in /data/travel,"Costa Rica Travel,Rio Celeste Costa Rica,Dream Vacations,Vacation Spots,Vacation Travel,Travel Pictures,Travel Photos,Fortuna Costa Rica,Costa Rica Pictures","14 Amazing Things To Do In Costa Rica | Volcanoes, Waterfalls, Wildlife And More",927c4658-cc3f-4b92-9b5c-70743d0c238d
diy-and-crafts,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
christmas,Here are the best DIY Christmas Centerpieces ideas perfect for your Christmas & holiday season home decor. From Christmas Vignettes to Table Centerpieces.,1,500k,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,2418,image,HikenDip,Local save in /data/christmas,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike n Dip,da8745a6-5160-46c4-877d-181d50a729fd
quotes,summcoco gives you inspiration for the women fashion trends you want. Thinking about a new look or lifestyle? This is your ultimate resource to get the hottest trends. 45 Top Li…,1,306k,https://i.pinimg.com/originals/bb/c0/e6/bbc0e6a797079505f11ac12bcb0b8c66.jpg,7922,image,"Sumcoco | Decor Ideas, Hairstyles, Nails Fashion Advice",Local save in /data/quotes,"Life Quotes Love,Inspirational Quotes About Love,Mood Quotes,Motivational Quotes,Tears Quotes,Quotes About Sadness,Deep Quotes About Life,Quotes Quotes,Quote Life",45 Top Life Quotes School Did Not Teach You,a584581c-1b38-4731-a1c

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03 05:41:59
Saint Kitts and Nevis,10663,-27.3474,-162.83,2019-07-25 18:53:51
Antigua and Barbuda,7922,-88.0974,-172.052,2021-01-27 09:14:19
Antigua and Barbuda,8606,-88.0974,-172.052,2021-03-28 14:54:07
Dominican Republic,9979,14.9967,-120.682,2018-07-18 19:01:46
Christmas Island,1704,-56.8702,-93.6232,2018-11-29 04:46:39
Papua New Guinea,7790,-43.692,64.9839,2018-07-31 08:19:15
American Samoa,9759,-77.9744,-106.258,2017-12-30 13:05:49


age,date_joined,first_name,ind,last_name
20,2015-12-01 15:08:31,Christopher,5076,Butler
39,2017-07-19 07:12:04,Michelle,7790,Gutierrez
21,2015-11-10 09:27:42,Andrea,8731,Alexander
24,2016-03-31 20:56:39,Austin,8887,Rodriguez
36,2015-12-20 16:38:13,Michelle,4315,Prince
32,2017-10-10 20:09:33,Christian,10625,Lang
22,2016-02-11 20:46:04,Jennifer,9672,Hudson
32,2016-04-02 03:51:23,Brittany,1313,Jones
34,2016-12-22 00:02:02,Thomas,10794,Turner
20,2016-01-07 19:49:22,David,2959,Griffith


# Unmount S3 bucket

In [ ]:
# unmount the bucket from the filestore
dbutils.fs.unmount("/mnt/user-1215be80977f-bucket")

/mnt/user-1215be80977f-bucket has been unmounted.
Out[8]: True